## Home assignment 05: Bagging and OOB score

Please, fill the lines in the code below.
This is a simplified version of `BaggingRegressor` from `sklearn`. Please, notice, that `sklearn` API is **not preserved**.

Your algorithm should be able to train different instances of the same model class on bootstrapped datasets and to provide [OOB score](https://en.wikipedia.org/wiki/Out-of-bag_error) for the training set.

The model should be passed as model class with no explicit parameters and no parentheses.

Example:
```
import numpy as np
from sklearn.linear_model import LinearRegression

bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
bagging_regressor.fit(LinearRegression, X, y)

```

In [1]:
import numpy as np

In [3]:
# class SimplifiedBaggingRegressor:
#     def __init__(self, num_bags, oob=False):
#         self.num_bags = num_bags
#         self.oob = oob
        
#     def _generate_splits(self, data: np.ndarray):
#         '''
#         Generate indices for every bag and store in self.indices_list list
#         '''
#         self.indices_list = []
#         data_length = len(data)
#         for bag in range(self.num_bags):
#             self.indices_list.append(np.random.choice(data_length, data_length))
        
#     def fit(self, model_constructor, data, target):
#         '''
#         Fit model on every bag.
#         Model constructor with no parameters (and with no ()) is passed to this function.
        
#         example:
        
#         bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
#         bagging_regressor.fit(LinearRegression, X, y)
#         '''
#         self.data = None
#         self.target = None
#         self._generate_splits(data)
#         assert len(set(list(map(len, self.indices_list)))) == 1, 'All bags should be of the same length!'
#         assert list(map(len, self.indices_list))[0] == len(data), 'All bags should contain `len(data)` number of elements!'
#         self.models_list = []
#         for bag in range(self.num_bags):
#             model = model_constructor()
#             data_bag, target_bag = data[self.indices_list[bag]], target[self.indices_list[bag]] # Your Code Here
#             self.models_list.append(model.fit(data_bag, target_bag)) # store fitted models here
#         if self.oob:
#             self.data = data
#             self.target = target
        
#     def predict(self, data):
#         '''
#         Get average prediction for every object from passed dataset
#         '''
#         preds = []
#         for model in self.models_list:
#             preds.append(model.predict(data))

#         return np.mean(preds, axis=0)
    
#     def _get_oob_predictions_from_every_model(self):
#         '''
#         Generates list of lists, where list i contains predictions for self.data[i] object
#         from all models, which have not seen this object during training phase
#         '''
#         # list_of_predictions_lists = [[] for _ in range(len(self.data))]

#         # for (bag, model) in zip(self.indices_list, self.models_list):
#         #     oob_indexes = np.setdiff1d(range(len(self.data)), bag)
#         #     pred = model.predict(self.data[oob_indexes])
#         #     list_of_predictions_lists.append(pred)
#         #     list_of_predictions_indexes.append(oob_indexes)
#         #     print(pred[:10])
#         #     print(oob_indexes[:10])

#         # self.list_of_predictions_indexes = np.array(list_of_predictions_indexes, dtype=object)
#         # self.list_of_predictions_lists = np.array(list_of_predictions_lists, dtype=object)
#         pass
    
#     def _get_averaged_oob_predictions(self):
#         '''
#         Compute average prediction for every object from training set.
#         If object has been used in all bags on training phase, return None instead of prediction
#         '''
        
#         preds = np.zeros(len(self.data))
#         preds_count = np.zeros(len(self.data))

#         for (bag, model) in zip(self.indices_list, self.models_list):
#             oob_indexes = np.setdiff1d(range(len(self.data)), bag)
#             pred = model.predict(self.data[oob_indexes])
#             preds[oob_indexes] += pred
#             preds_count[oob_indexes] += 1

#         self.oob_predictions = np.divide(preds, preds_count) # Your Code Here
        
        
#     def OOB_score(self):
#         '''
#         Compute mean square error for all objects, which have at least one prediction
#         '''
#         self._get_averaged_oob_predictions()
#         return np.power(self.oob_predictions - self.target, 2)

### Local tests:

In [1]:
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm
from bagging import SimplifiedBaggingRegressor
import numpy as np

#### Simple tests:

In [2]:
for _ in tqdm(range(100)):
    X = np.random.randn(2000, 10)
    y = np.mean(X, axis=1)
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    assert np.mean((predictions - y)**2) < 1e-6, 'Linear dependency should be fitted with almost zero error!'
    assert bagging_regressor.oob, 'OOB feature must be turned on'
    oob_score = bagging_regressor.OOB_score()
    assert oob_score < 1e-6, 'OOB error for linear dependency should be also close to zero!'
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 0.1, 'Probability of missing a bag should be close to theoretical value!'
    
print('Simple tests done!')

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

#### Medium tests

In [3]:
for _ in tqdm(range(10)):
    X = np.random.randn(200, 150)
    y = np.random.randn(len(X))
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=20, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    average_train_error = np.mean((predictions - y)**2)
    assert bagging_regressor.oob, 'OOB feature must be turned on'
    oob_score = bagging_regressor.OOB_score()
    assert oob_score > average_train_error, 'OOB error must be higher than train error due to overfitting!'
    print(np.mean(list(map(len, bagging_regressor.list_of_predictions_lists))), bagging_regressor.num_bags - 1/np.exp(1))
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 0.1, 'Probability of missing a bag should be close to theoretical value!'
    
print('Medium tests done!')

  0%|          | 0/10 [00:00<?, ?it/s]

6.763636363636364 19.632120558828557
6.586363636363636 19.632120558828557
6.681818181818182 19.632120558828557
6.622727272727273 19.632120558828557
6.659090909090909 19.632120558828557
6.5 19.632120558828557
6.5 19.632120558828557
6.668181818181818 19.632120558828557
6.6045454545454545 19.632120558828557
6.7272727272727275 19.632120558828557
Medium tests done!


#### Complex tests:

In [3]:
for _ in tqdm(range(10)):
    X = np.random.randn(2000, 15)
    y = np.random.randn(len(X))
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=100, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    oob_score = bagging_regressor.OOB_score()
    print(np.mean(list(map(len, bagging_regressor.list_of_predictions_lists))), bagging_regressor.num_bags - 1/np.exp(1))
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 1e-2, 'Probability of missing a bag should be close to theoretical value!'
    
print('Complex tests done!')

  0%|          | 0/10 [00:00<?, ?it/s]

36.802 99.63212055882856


KeyboardInterrupt: 

In [5]:
np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)

-0.07019372688572806

Great job! Please, save `SimplifiedBaggingRegressor` to  `bagging.py` and submit your solution to the grading system!